In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
import os
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
FGSM_Images_ModelRun_class_ids = []
# Specify the path to the folder containing the images
folder_path = "C:/Users/DandaC8719/Documents/capstone_overall_code/original_incorrect_predictions_folder(PGD)"
# Step 3: Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.JPEG'):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Use torchvision to read the image
        img = read_image(image_path)

        # Step 4: Apply inference preprocessing transforms
        batch = preprocess(img).unsqueeze(0)

        # Step 5: Use the model and print the predicted category
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        FGSM_Images_ModelRun_class_ids.append(class_id)
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]
        print(f"{filename}: {category_name}: {100 * score:.1f}%")
#print("Class ID's of FGSM images after running them through the model",FGSM_Images_ModelRun_class_ids)
print(len(FGSM_Images_ModelRun_class_ids)) 

ILSVRC2012_val_00000001.JPEG: sea snake: 25.0%
ILSVRC2012_val_00000051.JPEG: bee eater: 67.6%
ILSVRC2012_val_00000056.JPEG: cab: 38.2%
ILSVRC2012_val_00000064.JPEG: American chameleon: 22.4%
ILSVRC2012_val_00000068.JPEG: vestment: 30.2%
ILSVRC2012_val_00000079.JPEG: otterhound: 63.2%
ILSVRC2012_val_00000093.JPEG: gorilla: 52.1%
ILSVRC2012_val_00000094.JPEG: catamaran: 15.7%
ILSVRC2012_val_00000097.JPEG: wood rabbit: 31.6%
ILSVRC2012_val_00000099.JPEG: strawberry: 79.1%
ILSVRC2012_val_00000100.JPEG: bighorn: 46.9%
ILSVRC2012_val_00000109.JPEG: loudspeaker: 37.7%
ILSVRC2012_val_00000129.JPEG: hotdog: 80.8%
ILSVRC2012_val_00000149.JPEG: hand-held computer: 43.3%
ILSVRC2012_val_00000177.JPEG: red-backed sandpiper: 49.3%
ILSVRC2012_val_00000178.JPEG: Siberian husky: 27.9%
ILSVRC2012_val_00000193.JPEG: lakeside: 23.1%
ILSVRC2012_val_00000202.JPEG: bonnet: 4.6%
ILSVRC2012_val_00000212.JPEG: violin: 58.1%
ILSVRC2012_val_00000215.JPEG: car mirror: 65.6%
ILSVRC2012_val_00000220.JPEG: pug: 64.7%


In [4]:
input_directory = "./adversarial_images_folder(PGD)"
output_directory = "./Blurred_images(MedianBlur)PGD"
#Median Blur
os.makedirs(output_directory, exist_ok=True)

# Iterate over input images
input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

for input_image_path in input_images:
    # Load the image using cv2.imread()
    image_BGR = cv2.imread(input_image_path)
    
    # Apply median blur
    median_blur = cv2.medianBlur(image_BGR, 5)  # Adjust the kernel size (5 in this case)
    
    # Save the blurred image
    output_image_path = os.path.join(output_directory, os.path.basename(input_image_path))
    cv2.imwrite(output_image_path, median_blur)

In [5]:
input_directory = "./adversarial_images_folder(PGD)"
output_directory = "./Blurred_images(Gaussian)PGD"
# GaussianBlur
os.makedirs(output_directory, exist_ok=True)
input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]   
    
# Define parameters for Gaussian blur
kernel_size = (5, 5)  # Adjust the kernel size as needed

# Define parameters for contrast enhancement
alpha = 1.5  # Adjust the contrast factor as needed
beta = 0  # Adjust the brightness factor as needed

# Apply preprocessing to each input image
for input_image_path in input_images:
    # Load the image using cv2.imread()
    image_BGR = cv2.imread(input_image_path)
    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image_BGR, kernel_size, 0)
    # Apply contrast enhancement
    enhanced_image = cv2.convertScaleAbs(blurred_image, alpha=alpha, beta=beta)
    # Save the preprocessed image
    output_image_path = os.path.join(output_directory, os.path.basename(input_image_path))
    cv2.imwrite(output_image_path, enhanced_image)


In [6]:
input_directory = "./adversarial_images_folder(PGD)"
output_directory = "./Blurred_images(Bilateral)PGD"
#BilateralBlur
os.makedirs(output_directory, exist_ok=True)
input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]
for input_image_path in input_images:
    # Load the image using cv2.imread()
    image_BGR = cv2.imread(input_image_path)
    # Convert BGR to RGB
    image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
    # Apply bilateral blur
    bilateral_blur = cv2.bilateralFilter(image_RGB, 23, 51, 51)
    # Save the blurred image
    output_image_path = os.path.join(output_directory, os.path.basename(input_image_path))
    cv2.imwrite(output_image_path, bilateral_blur)

In [7]:
input_directory = "./adversarial_images_folder(PGD)"
output_directory = "./Blurred_images(Affine)PGD"
#Affine Transformation
# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)
# Load input images
input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Define the affine transformation matrix
matrix = np.float32([[1, 0, 50],   # Translation in x-direction
                     [0, 1, -30]])  # Translation in y-direction
# Apply affine transformation to each input image
for input_image_path in input_images:
    # Load the image using cv2.imread()
    image_BGR = cv2.imread(input_image_path)
    # Apply affine transformation
    transformed_image = cv2.warpAffine(image_BGR, matrix, (image_BGR.shape[1], image_BGR.shape[0]))
    # Save the preprocessed image
    output_image_path = os.path.join(output_directory, os.path.basename(input_image_path))
    cv2.imwrite(output_image_path, transformed_image)
    

In [8]:
import os
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
import torch
from torchvision import models

# Load resnet50
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
Median_BlurredImages_ModelRun_class_ids = []
 
# Specify the path to the folder containing the images
folder_path = "C:/Users/DandaC8719/Documents/capstone_overall_code/Blurred_images(MedianBlur)PGD"

# Step 3: Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.JPEG'):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Use torchvision to read the image
        img = read_image(image_path)

        # Step 4: Apply inference preprocessing transforms
        batch = preprocess(img).unsqueeze(0)

        # Step 5: Use the model and print the predicted category
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        Median_BlurredImages_ModelRun_class_ids.append(class_id)
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]
        print(f"{filename}: {category_name}: {100 * score:.1f}%")
#print("Class ID's of Bilateral blurred images after running them through the model",Bilateral_BlurredImages_ModelRun_class_ids)
print(len(Median_BlurredImages_ModelRun_class_ids))

perturbed_ILSVRC2012_val_00000001.JPEG: sea snake: 39.9%
perturbed_ILSVRC2012_val_00000051.JPEG: bee eater: 38.8%
perturbed_ILSVRC2012_val_00000056.JPEG: cab: 22.6%
perturbed_ILSVRC2012_val_00000064.JPEG: American chameleon: 23.6%
perturbed_ILSVRC2012_val_00000068.JPEG: vestment: 10.4%
perturbed_ILSVRC2012_val_00000079.JPEG: otterhound: 60.8%
perturbed_ILSVRC2012_val_00000093.JPEG: gorilla: 20.3%
perturbed_ILSVRC2012_val_00000094.JPEG: schooner: 15.0%
perturbed_ILSVRC2012_val_00000097.JPEG: wood rabbit: 29.7%
perturbed_ILSVRC2012_val_00000099.JPEG: strawberry: 89.8%
perturbed_ILSVRC2012_val_00000100.JPEG: ibex: 14.0%
perturbed_ILSVRC2012_val_00000109.JPEG: loudspeaker: 37.8%
perturbed_ILSVRC2012_val_00000129.JPEG: hotdog: 56.5%
perturbed_ILSVRC2012_val_00000149.JPEG: iPod: 43.1%
perturbed_ILSVRC2012_val_00000177.JPEG: red-backed sandpiper: 52.6%
perturbed_ILSVRC2012_val_00000178.JPEG: Siberian husky: 27.4%
perturbed_ILSVRC2012_val_00000193.JPEG: lakeside: 38.6%
perturbed_ILSVRC2012_val

In [9]:
import os
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
import torch
from torchvision import models

# Load resnet50
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
Gaussian_BlurredImages_ModelRun_class_ids = []
 
# Specify the path to the folder containing the images
folder_path = "C:/Users/DandaC8719/Documents/capstone_overall_code/Blurred_images(Gaussian)PGD"

# Step 3: Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.JPEG'):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Use torchvision to read the image
        img = read_image(image_path)

        # Step 4: Apply inference preprocessing transforms
        batch = preprocess(img).unsqueeze(0)

        # Step 5: Use the model and print the predicted category
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        Gaussian_BlurredImages_ModelRun_class_ids.append(class_id)
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]
        print(f"{filename}: {category_name}: {100 * score:.1f}%")
#print("Class ID's of Bilateral blurred images after running them through the model",Bilateral_BlurredImages_ModelRun_class_ids)
print(len(Gaussian_BlurredImages_ModelRun_class_ids))

perturbed_ILSVRC2012_val_00000001.JPEG: rock python: 31.2%
perturbed_ILSVRC2012_val_00000051.JPEG: bee eater: 50.7%
perturbed_ILSVRC2012_val_00000056.JPEG: cab: 34.4%
perturbed_ILSVRC2012_val_00000064.JPEG: American chameleon: 26.0%
perturbed_ILSVRC2012_val_00000068.JPEG: vestment: 49.8%
perturbed_ILSVRC2012_val_00000079.JPEG: otterhound: 52.2%
perturbed_ILSVRC2012_val_00000093.JPEG: gorilla: 21.8%
perturbed_ILSVRC2012_val_00000094.JPEG: yawl: 13.8%
perturbed_ILSVRC2012_val_00000097.JPEG: wood rabbit: 26.1%
perturbed_ILSVRC2012_val_00000099.JPEG: strawberry: 88.6%
perturbed_ILSVRC2012_val_00000100.JPEG: ibex: 20.2%
perturbed_ILSVRC2012_val_00000109.JPEG: loudspeaker: 46.0%
perturbed_ILSVRC2012_val_00000129.JPEG: hotdog: 28.3%
perturbed_ILSVRC2012_val_00000149.JPEG: hand-held computer: 13.8%
perturbed_ILSVRC2012_val_00000177.JPEG: red-backed sandpiper: 59.8%
perturbed_ILSVRC2012_val_00000178.JPEG: Eskimo dog: 34.4%
perturbed_ILSVRC2012_val_00000193.JPEG: lakeside: 35.2%
perturbed_ILSVRC

In [10]:
import os
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
import torch
from torchvision import models

# Load resnet50
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
Bilateral_BlurredImages_ModelRun_class_ids = []
 
# Specify the path to the folder containing the images
folder_path = "C:/Users/DandaC8719/Documents/capstone_overall_code/Blurred_images(Bilateral)PGD"

# Step 3: Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.JPEG'):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Use torchvision to read the image
        img = read_image(image_path)

        # Step 4: Apply inference preprocessing transforms
        batch = preprocess(img).unsqueeze(0)

        # Step 5: Use the model and print the predicted category
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        Bilateral_BlurredImages_ModelRun_class_ids.append(class_id)
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]
        print(f"{filename}: {category_name}: {100 * score:.1f}%")
#print("Class ID's of Bilateral blurred images after running them through the model",Bilateral_BlurredImages_ModelRun_class_ids)
print(len(Bilateral_BlurredImages_ModelRun_class_ids))

perturbed_ILSVRC2012_val_00000001.JPEG: rock python: 18.5%
perturbed_ILSVRC2012_val_00000051.JPEG: bee eater: 38.6%
perturbed_ILSVRC2012_val_00000056.JPEG: minivan: 13.7%
perturbed_ILSVRC2012_val_00000064.JPEG: green lizard: 11.8%
perturbed_ILSVRC2012_val_00000068.JPEG: vestment: 7.9%
perturbed_ILSVRC2012_val_00000079.JPEG: West Highland white terrier: 12.5%
perturbed_ILSVRC2012_val_00000093.JPEG: siamang: 20.3%
perturbed_ILSVRC2012_val_00000094.JPEG: yawl: 9.5%
perturbed_ILSVRC2012_val_00000097.JPEG: wood rabbit: 42.5%
perturbed_ILSVRC2012_val_00000099.JPEG: strawberry: 62.3%
perturbed_ILSVRC2012_val_00000100.JPEG: bighorn: 23.9%
perturbed_ILSVRC2012_val_00000109.JPEG: loudspeaker: 38.1%
perturbed_ILSVRC2012_val_00000129.JPEG: hotdog: 12.9%
perturbed_ILSVRC2012_val_00000149.JPEG: hand-held computer: 27.9%
perturbed_ILSVRC2012_val_00000177.JPEG: red-backed sandpiper: 33.9%
perturbed_ILSVRC2012_val_00000178.JPEG: Siberian husky: 30.4%
perturbed_ILSVRC2012_val_00000193.JPEG: boathouse: 1

In [11]:
import os
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
import torch
from torchvision import models

# Load resnet50
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
Affine_BlurredImages_ModelRun_class_ids = []
 
# Specify the path to the folder containing the images
folder_path = "C:/Users/DandaC8719/Documents/capstone_overall_code/Blurred_images(Affine)PGD"

# Step 3: Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.JPEG'):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Use torchvision to read the image
        img = read_image(image_path)

        # Step 4: Apply inference preprocessing transforms
        batch = preprocess(img).unsqueeze(0)

        # Step 5: Use the model and print the predicted category
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        Affine_BlurredImages_ModelRun_class_ids.append(class_id)
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]
        print(f"{filename}: {category_name}: {100 * score:.1f}%")
#print("Class ID's of Bilateral blurred images after running them through the model",Bilateral_BlurredImages_ModelRun_class_ids)
print(len(Affine_BlurredImages_ModelRun_class_ids))

perturbed_ILSVRC2012_val_00000001.JPEG: rock python: 36.3%
perturbed_ILSVRC2012_val_00000051.JPEG: bee eater: 48.3%
perturbed_ILSVRC2012_val_00000056.JPEG: cab: 27.5%
perturbed_ILSVRC2012_val_00000064.JPEG: green lizard: 23.9%
perturbed_ILSVRC2012_val_00000068.JPEG: vestment: 23.9%
perturbed_ILSVRC2012_val_00000079.JPEG: otterhound: 60.7%
perturbed_ILSVRC2012_val_00000093.JPEG: gorilla: 49.9%
perturbed_ILSVRC2012_val_00000094.JPEG: schooner: 12.7%
perturbed_ILSVRC2012_val_00000097.JPEG: wood rabbit: 23.6%
perturbed_ILSVRC2012_val_00000099.JPEG: strawberry: 58.1%
perturbed_ILSVRC2012_val_00000100.JPEG: bighorn: 14.2%
perturbed_ILSVRC2012_val_00000109.JPEG: loudspeaker: 31.1%
perturbed_ILSVRC2012_val_00000129.JPEG: hotdog: 17.6%
perturbed_ILSVRC2012_val_00000149.JPEG: hand-held computer: 23.4%
perturbed_ILSVRC2012_val_00000177.JPEG: red-backed sandpiper: 48.9%
perturbed_ILSVRC2012_val_00000178.JPEG: Siberian husky: 21.7%
perturbed_ILSVRC2012_val_00000193.JPEG: drilling platform: 15.9%
pe

In [12]:
# Assuming you have already executed the code snippets to obtain the arrays

# Create sets from the arrays for efficient comparison
median_set = (Median_BlurredImages_ModelRun_class_ids)
PGD_set = (FGSM_Images_ModelRun_class_ids)

# Ensure both lists have the same length
if len(Median_BlurredImages_ModelRun_class_ids) != len(FGSM_Images_ModelRun_class_ids):
    print("Lists have different lengths. Cannot compare.")
else:
    # Initialize a counter for matching elements
    matching_count = 0

    # Iterate over the indices of the lists
    for i in range(len(Median_BlurredImages_ModelRun_class_ids)):
        # Compare elements at the same index in both lists
        if Median_BlurredImages_ModelRun_class_ids[i] == FGSM_Images_ModelRun_class_ids[i]:
            # If they match, increment the matching count
            matching_count += 1

    print("Number of matching elements:", matching_count)

Number of matching elements: 3593


In [13]:
# Assuming you have already executed the code snippets to obtain the arrays

# Create sets from the arrays for efficient comparison
gaussian_set = (Gaussian_BlurredImages_ModelRun_class_ids)
PGD_set = (FGSM_Images_ModelRun_class_ids)

# Ensure both lists have the same length
if len(Gaussian_BlurredImages_ModelRun_class_ids) != len(FGSM_Images_ModelRun_class_ids):
    print("Lists have different lengths. Cannot compare.")
else:
    # Initialize a counter for matching elements
    matching_count = 0

    # Iterate over the indices of the lists
    for i in range(len(Gaussian_BlurredImages_ModelRun_class_ids)):
        # Compare elements at the same index in both lists
        if Gaussian_BlurredImages_ModelRun_class_ids[i] == FGSM_Images_ModelRun_class_ids[i]:
            # If they match, increment the matching count
            matching_count += 1

    print("Number of matching elements:", matching_count)


Number of matching elements: 3868


In [14]:
# Assuming you have already executed the code snippets to obtain the arrays

# Create sets from the arrays for efficient comparison
bilateral_set = (Bilateral_BlurredImages_ModelRun_class_ids)
PGD_set = (FGSM_Images_ModelRun_class_ids)

# Ensure both lists have the same length
if len(Bilateral_BlurredImages_ModelRun_class_ids) != len(FGSM_Images_ModelRun_class_ids):
    print("Lists have different lengths. Cannot compare.")
else:
    # Initialize a counter for matching elements
    matching_count = 0

    # Iterate over the indices of the lists
    for i in range(len(Bilateral_BlurredImages_ModelRun_class_ids)):
        # Compare elements at the same index in both lists
        if Bilateral_BlurredImages_ModelRun_class_ids[i] == FGSM_Images_ModelRun_class_ids[i]:
            # If they match, increment the matching count
            matching_count += 1

    print("Number of matching elements:", matching_count)


Number of matching elements: 2792


In [15]:
# Assuming you have already executed the code snippets to obtain the arrays

# Create sets from the arrays for efficient comparison
affine_set = (Affine_BlurredImages_ModelRun_class_ids)
PGD_set = (FGSM_Images_ModelRun_class_ids)

# Ensure both lists have the same length
if len(Affine_BlurredImages_ModelRun_class_ids) != len(FGSM_Images_ModelRun_class_ids):
    print("Lists have different lengths. Cannot compare.")
else:
    # Initialize a counter for matching elements
    matching_count = 0

    # Iterate over the indices of the lists
    for i in range(len(Affine_BlurredImages_ModelRun_class_ids)):
        # Compare elements at the same index in both lists
        if Affine_BlurredImages_ModelRun_class_ids[i] == FGSM_Images_ModelRun_class_ids[i]:
            # If they match, increment the matching count
            matching_count += 1

    print("Number of matching elements:", matching_count)


Number of matching elements: 4075
